# Project runner
Imports the latest project version and runs the specified workflow

In [8]:
import nuclio

In [61]:
%nuclio config spec.build.baseImage = "mlrun/mlrun"

%nuclio: setting spec.build.baseImage to 'mlrun/mlrun'


In [9]:
# nuclio: start-code

In [10]:
from mlrun import load_project
from mlrun import mlconf
import json
import os

In [22]:
def init_context(context):
    setattr(context, 'hub_url', os.getenv('hub_url', None))

In [37]:
def handler(context, event):
    """Imports the latest project version and runs the 
    specified workflow
    """
    context.logger.info('Pulling project and workflow details')
    if isinstance(event.body, dict):
        details = event.body
    else:
        details = json.loads(event.body)
    context.logger.info(details)
    project_url = details['project_url']
    workflow = details['workflow']
    artifact_path = details.get('artifact_path', os.environ.get('artifact_path', None))
    hub_url = details.get('hub_url', context.hub_url)

    if hub_url:
        mlconf.hub_url = hub_url

    project= load_project(os.path.abspath('./loaded_project'), url=project_url)
    project.run(name=workflow,
                arguments={},
                artifact_path=artifact_path,
                watch=False)


In [32]:
# nuclio: end-code

In [63]:
import json
runner_event = {'project_url': '/User/demo-network-operations/project.yaml',
                'workflow': 'main',
                'hub_url': '/User/functions/{name}/function.yaml',
                 'artifact_path': '/User/functions/project_runner/artifacts/'}

In [46]:
init_context(context)
event = nuclio.Event(body=json.dumps(runner_event))
out = handler(context, event)
out

Python> 2020-07-01 14:36:45,368 [info] <class 'dict'>
Python> 2020-07-01 14:36:45,369 [info] {'project_url': '/User/demo-network-operations/project.yaml', 'workflow': 'main', 'hub_url': '/User/functions/{name}/function.yaml', 'artifact_path': '/User/functions/project_runner/artifacts/'}


/conda/lib/python3.6/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "JsonArray" based on the value "['cpu_utilization', 'throughput', 'packet_loss', 'latency']".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/conda/lib/python3.6/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "JsonArray" based on the value "['mean', 'sum', 'std', 'var', 'min', 'max', 'median']".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/conda/lib/python3.6/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "Integer" based on the value "20".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/conda/lib/python3.6/site-packages/kfp/components/_data_passing.py:168: UserWarning: Mi

[mlrun] 2020-07-01 14:36:46,646 Pipeline run id=cf85ec1b-2df7-403c-b7b4-b9bdb8fcf92f, check UI or DB for progress


# Deployment

In [48]:
from mlrun import code_to_function, mount_v3io
from nuclio.triggers import CronTrigger

In [89]:
# Saving the function for import via hub://project_runner
fn = code_to_function(name='project-runner',
                      kind='nuclio')
fn.spec.description = 'Nuclio based - Cron scheduler for running your MLRun projects'
fn.metadata.categories =  ["utils"]
fn.metadata.labels = {'author': 'orz'}
fn.spec.maxReplicas = 1
fn.export('function.yaml')

[mlrun] 2020-07-02 09:30:36,014 function spec saved to path: function.yaml


### How to call from your project?
> **After** importing the function

In [90]:
cron_string = '* * 1 * *' # Regular cron string as in https://pypi.org/project/croniter/

# Set defaults
fn.set_envs({'artifact_path': '/User/functions/project_runner/artifacts/',
             'hub_url': '/User/functions/{name}/function.yaml'})

# Set project and workflow event
runner_event = {'project_url': '/User/demo-network-operations/project.yaml',
                'workflow': 'main'}

# Add as a trigger
fn.add_trigger('cron', 
               CronTrigger(schedule=cron_string,
                           body=json.dumps(runner_event),
                           headers={'X-Nuclio-Target': 'project-runner'}))

# Add mount for access to the different directories
fn.apply(mount_v3io())

In [ ]:
fn.deploy()

[mlrun] 2020-07-02 09:31:16,905 deploy started
[nuclio] 2020-07-02 09:31:19,021 (info) Build complete
